# Housing Prices Competition for Kaggle Learn Users

Housing Prices Competition for Kaggle Learn Users là một bộ dữ liệu được cung cấp bởi Kaggle, một nền tảng nổi tiếng về các cuộc thi về khoa học dữ liệu và máy học. Bộ dữ liệu này bao gồm thông tin về giá nhà ở Ames, Iowa, Hoa Kỳ.

Bộ dữ liệu này bao gồm hai tập dữ liệu, một tập dữ liệu huấn luyện và một tập dữ liệu kiểm tra. Tập dữ liệu huấn luyện bao gồm 1460 mẫu và 80 biến đầu vào, bao gồm các thông tin về kích thước, vị trí, tiện nghi và trạng thái của ngôi nhà. Biến đầu ra là giá trị của ngôi nhà. Tập dữ liệu kiểm tra bao gồm 1459 mẫu và cũng bao gồm các biến đầu vào nhưng không có biến đầu ra.

Bộ dữ liệu này được sử dụng để đào tạo các mô hình dự đoán giá nhà dựa trên các đặc trưng của ngôi nhà. Nhiệm vụ của các thí sinh là tìm ra một mô hình tốt nhất để dự đoán giá trị của các ngôi nhà trong tập dữ liệu kiểm tra. Bộ dữ liệu này là một trong những bộ dữ liệu phổ biến và được sử dụng nhiều trong lĩnh vực dự đoán giá nhà và phân tích dữ liệu.


## Table of Contents
- [1 - Cài đặt dữ liệu từ Kaggle](#1)
- [2 - Lựa chọn và chuẩn hóa dữ liệu cần cho việc training](#2)
- [3 - Thực hiện trainning dựa trên model của Tensorflow Keras](#3)
- [4 - Sử dụng Hyperparameter Tuning để cải tiến model](#4)
- [5 - Cải tiến Hyperparameter Tuning](#5)
- [6 - Cải tiến bằng cách dùng Hyperband](#6)

<a name='1'></a>
## 1 - Cài đặt dữ liệu từ Kaggle

In [1]:
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import numpy as np
# tạo các tham số môi trường cho api kaggle để kết nối tải file
os.environ['KAGGLE_USERNAME'] = "sonsiho"
os.environ['KAGGLE_KEY'] = "c728fa0b1cd641b8371cc52046933a59"

In [3]:
!kaggle competitions download -c home-data-for-ml-course #tải gói data cho bài tutorial từ kaggle dưới dạng zip file
!mkdir kaggle #tạo folder để unzip file vừa tải
!unzip /content/home-data-for-ml-course.zip -d kaggle #tiến hành unzip vào folder kaggle

  0% 0.00/386k [00:00<?, ?B/s]
100% 386k/386k [00:00<00:00, 87.1MB/s]
Archive:  /content/home-data-for-ml-course.zip
  inflating: kaggle/data_description.txt  
  inflating: kaggle/sample_submission.csv  
  inflating: kaggle/sample_submission.csv.gz  
  inflating: kaggle/test.csv         
  inflating: kaggle/test.csv.gz      
  inflating: kaggle/train.csv        
  inflating: kaggle/train.csv.gz     


<a name='2'></a>
## 2 - Lựa chọn và chuẩn hóa dữ liệu cần cho việc training

In [4]:
# Đọc dữ liệu từ file train.csv
train_data = pd.read_csv("kaggle/train.csv")
# Hiển thị 5 dòng đầu tiên của dữ liệu
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# xem thucộ tính của các trường dữ liệu
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
# dữ liệu có 1460 dòng và 81 trường dữ liệu
train_data.shape

(1460, 81)

In [7]:
# lựa chọn các trường dữ liệu
train_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [8]:
features = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
X= train_data[features]
y=train_data['SalePrice']

In [9]:
X.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,TotRmsAbvGrd,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,856,854,0,1710,...,8,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,1262,0,0,1262,...,6,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,920,866,0,1786,...,6,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,961,756,0,1717,...,7,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,1145,1053,0,2198,...,9,192,84,0,0,0,0,0,12,2008


In [10]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [11]:
# kiểm tra dữ liệu rỗng
X.isnull().sum().sort_values(ascending = False)

MSSubClass       0
LotArea          0
MoSold           0
MiscVal          0
PoolArea         0
ScreenPorch      0
3SsnPorch        0
EnclosedPorch    0
OpenPorchSF      0
WoodDeckSF       0
TotRmsAbvGrd     0
KitchenAbvGr     0
BedroomAbvGr     0
HalfBath         0
FullBath         0
GrLivArea        0
LowQualFinSF     0
2ndFlrSF         0
1stFlrSF         0
YearRemodAdd     0
YearBuilt        0
OverallCond      0
OverallQual      0
YrSold           0
dtype: int64

<a name='3'></a>
## 3 - Thực hiện trainning dựa trên model của Tensorflow Keras

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu đầu vào
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
X_train

array([[-0.8667643 , -0.21289571, -0.82044456, ..., -0.09274033,
        -0.13341669,  1.65006527],
       [ 0.07410996, -0.26524463, -0.08893368, ..., -0.09274033,
        -0.5080097 ,  0.89367742],
       [-0.63154574, -0.17784146, -0.82044456, ..., -0.09274033,
        -0.5080097 ,  0.13728958],
       ...,
       [-0.8667643 , -0.23409563, -0.82044456, ..., -0.09274033,
        -0.88260272, -1.37548612],
       [-0.16110861, -0.28337613,  0.64257719, ..., -0.09274033,
        -0.13341669, -0.61909827],
       [ 1.48542135, -0.65139925,  0.64257719, ..., -0.09274033,
        -0.13341669,  0.89367742]])

In [14]:
X_test

array([[-0.8667643 , -0.21159396, -0.08893368, ..., -0.09274033,
        -1.63178875, -1.37548612],
       [ 0.07410996,  0.14564323,  1.37408806, ..., -0.09274033,
        -0.88260272,  1.65006527],
       [-0.63154574, -0.16082574, -0.82044456, ..., -0.09274033,
        -1.25719573,  1.65006527],
       ...,
       [ 0.07410996, -0.23158511,  0.64257719, ..., -0.09274033,
         1.36495537,  0.13728958],
       [ 0.30932853, -0.14929596, -1.55195543, ..., -0.09274033,
         1.36495537,  0.89367742],
       [-0.8667643 , -0.2389307 , -1.55195543, ..., -0.09274033,
         0.24117632,  0.89367742]])

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
import tensorflow.keras.backend as K

# tạo hàm r_squared để đánh giá model
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Khỏi tạo model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])

# Huấn luyện model trên tập huấn luyện
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/100
37/37 [==============================] - 1s 10ms/step - loss: 38885113856.0000 - mean_squared_error: 38885113856.0000 - root_mean_squared_error: 197193.0938 - mean_absolute_error: 181440.2344 - r_squared: -6.6821 - val_loss: 39652884480.0000 - val_mean_squared_error: 39652884480.0000 - val_root_mean_squared_error: 199130.3281 - val_mean_absolute_error: 178836.6562 - val_r_squared: -5.7364
Epoch 2/100
37/37 [==============================] - 0s 3ms/step - loss: 38883262464.0000 - mean_squared_error: 38883262464.0000 - root_mean_squared_error: 197188.3906 - mean_absolute_error: 181435.5000 - r_squared: -6.5882 - val_loss: 39650115584.0000 - val_mean_squared_error: 39650115584.0000 - val_root_mean_squared_error: 199123.3750 - val_mean_absolute_error: 178829.8125 - val_r_squared: -5.7359
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 38878752768.0000 - mean_squared_error: 38878752768.0000 - root_mean_squared_error: 197176.9531 - mean_absolute_error: 18

In [16]:
# chạy thử mô hình với tập kiểm tra
model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", model.metrics[1].result().numpy())
print("Root Mean Squared Error:", model.metrics[2].result().numpy())
print("Mean Absolute Error:", model.metrics[3].result().numpy())
print("R-squared:", model.metrics[4].result().numpy())

10/10 [==============================] - 0s 2ms/step - loss: 3961548288.0000 - mean_squared_error: 3961548288.0000 - root_mean_squared_error: 62940.8320 - mean_absolute_error: 48984.2695 - r_squared: 0.3301
Mean Squared Error: 3961548300.0
Root Mean Squared Error: 62940.832
Mean Absolute Error: 48984.27
R-squared: 0.3301308


### Kết luận

sau khi thực hiện train model và ta được kết quả:
- Mean Squared Error: 10546019000.0
- Root Mean Squared Error: 102693.81
- Mean Absolute Error: 92084.734
- R-squared: -0.92295706

<a name='4'></a>
## 4 - Sử dụng Hyperparameter Tuning để cải tiến model

In [17]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 6.4 MB/s eta 0:00:00


In [18]:
import tensorflow as tf
from kerastuner.tuners import RandomSearch
# tạo hàm build_model
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',
                                                             values=[1e-2, 1e-3, 1e-4])),
                  loss='mse',
                  metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])
    return model

# khỏi tạo tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=5,
    directory='kaggle',
    project_name='hoc_ml_mse11')

tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))

# Print best hyperparameters
print(tuner.get_best_hyperparameters()[0].values)

Trial 10 Complete [00h 00m 52s]
val_loss: 39644807987.2

Best val_loss So Far: 2677204684.8
Total elapsed time: 00h 09m 17s
{'units': 384, 'learning_rate': 0.01}


In [19]:
# lấy model tốt nhất
best_model = tuner.get_best_models(num_models=1)[0]

# chạy với tập train
best_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


Epoch 1/100
37/37 [==============================] - 2s 19ms/step - loss: 2750577152.0000 - mean_squared_error: 2750577152.0000 - root_mean_squared_error: 52445.9453 - mean_absolute_error: 36990.8008 - r_squared: 0.4495 - val_loss: 2488816896.0000 - val_mean_squared_error: 2488816896.0000 - val_root_mean_squared_error: 49888.0430 - val_mean_absolute_error: 37677.5547 - val_r_squared: 0.5993
Epoch 2/100
37/37 [==============================] - 0s 11ms/step - loss: 2686990336.0000 - mean_squared_error: 2686990336.0000 - root_mean_squared_error: 51836.1875 - mean_absolute_error: 36512.8906 - r_squared: 0.4210 - val_loss: 2431803392.0000 - val_mean_squared_error: 2431803392.0000 - val_root_mean_squared_error: 49313.3203 - val_mean_absolute_error: 37118.4961 - val_r_squared: 0.6101
Epoch 3/100
37/37 [==============================] - 0s 6ms/step - loss: 2620149504.0000 - mean_squared_error: 2620149504.0000 - root_mean_squared_error: 51187.3945 - mean_absolute_error: 36019.8750 - r_squared: 

In [20]:
# chạy thử mô hình với tập kiểm tra
best_model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model.metrics[3].result().numpy())
print("R-squared:", best_model.metrics[4].result().numpy())

10/10 [==============================] - 0s 2ms/step - loss: 1244914176.0000 - mean_squared_error: 1244914176.0000 - root_mean_squared_error: 35283.3398 - mean_absolute_error: 23082.8867 - r_squared: 0.8349
Mean Squared Error: 1244914200.0
Root Mean Squared Error: 35283.34
Mean Absolute Error: 23082.887
R-squared: 0.8348786


### Kết luận

sau khi sử dụng Hyperparameter Turning và ta được kết quả:
- Mean Squared Error: 1345812900.0
- Root Mean Squared Error: 36685.32
- Mean Absolute Error: 24583.262
- R-squared: 0.8155395


<a name='5'></a>
## 5 - Cải tiến Hyperparameter Tuning
- thay đổi giá trị max_trials=10

In [21]:
def build_model2(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout',
                                               min_value=0.0,
                                               max_value=0.5,
                                               step=0.1)))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',
                                                             values=[1e-2, 1e-3, 1e-4])),
                  loss='mse',
                  metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])
    return model
# Define tuner
tuner_2 = RandomSearch(
    build_model2,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=5,
    directory='kaggle',
    project_name='hoc_ml_mse11_turn2')

# Fit tuner to data
tuner_2.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))

# Print best hyperparameters
print(tuner_2.get_best_hyperparameters()[0].values)

Trial 10 Complete [00h 00m 54s]
val_loss: 3868988160.0

Best val_loss So Far: 2299276953.6
Total elapsed time: 00h 09m 06s
{'units': 512, 'dropout': 0.1, 'learning_rate': 0.01}


In [22]:
# lấy model tốt nhất
best_model_2 = tuner_2.get_best_models(num_models=1)[0]

# chạy với tập train
best_model_2.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
37/37 [==============================] - 1s 10ms/step - loss: 2477839872.0000 - mean_squared_error: 2477839872.0000 - root_mean_squared_error: 49777.9062 - mean_absolute_error: 34840.4844 - r_squared: 0.4984 - val_loss: 2264510208.0000 - val_mean_squared_error: 2264510208.0000 - val_root_mean_squared_error: 47586.8711 - val_mean_absolute_error: 35469.1523 - val_r_squared: 0.6431
Epoch 2/200
37/37 [==============================] - 0s 4ms/step - loss: 2405661952.0000 - mean_squared_error: 2405661952.0000 - root_mean_squared_error: 49047.5469 - mean_absolute_error: 34236.7852 - r_squared: 0.4951 - val_loss: 2204499712.0000 - val_mean_squared_error: 2204499712.0000 - val_root_mean_squared_error: 46952.1016 - val_mean_absolute_error: 34852.4023 - val_r_squared: 0.6533
Epoch 3/200
37/37 [==============================] - 0s 4ms/step - loss: 2357294336.0000 - mean_squared_error: 2357294336.0000 - root_mean_squared_error: 48551.9766 - mean_absolute_error: 33938.4023 - r_squared: 0

In [23]:
# chạy thử mô hình với tập kiểm tra
best_model_2.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model_2.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model_2.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model_2.metrics[3].result().numpy())
print("R-squared:", best_model_2.metrics[4].result().numpy())

10/10 [==============================] - 0s 3ms/step - loss: 1092072704.0000 - mean_squared_error: 1092072704.0000 - root_mean_squared_error: 33046.5234 - mean_absolute_error: 20757.2344 - r_squared: 0.8631
Mean Squared Error: 1092072700.0
Root Mean Squared Error: 33046.523
Mean Absolute Error: 20757.234
R-squared: 0.86305606


### Kết luận

Sau khi sử dụng thay đổi giá trị max_trials và ta được kết quả:

- Mean Squared Error: 1370459000.0
- Root Mean Squared Error: 37019.71
- Mean Absolute Error: 24677.445
- R-squared: 0.80979043




<a name='6'></a>
## 6 - Cải tiến bằng cách dùng Hyperband

In [24]:
from kerastuner import Hyperband
# Initialize the Hyperband tuner
tuner_3 = Hyperband(build_model2,
                     objective='val_loss',
                     max_epochs=50,
                     factor=3,
                     directory='kaggle',
                     project_name='hoc_ml_mse11_turn3')

# Perform the hyperparameter search
tuner_3.search(X_train, y_train, epochs=50, validation_split=0.2)

# Print best hyperparameters
print(tuner_2.get_best_hyperparameters()[0].values)


Trial 90 Complete [00h 00m 08s]
val_loss: 37839093760.0

Best val_loss So Far: 3153841920.0
Total elapsed time: 00h 05m 22s
{'units': 512, 'dropout': 0.1, 'learning_rate': 0.01}


In [25]:
# lấy model tốt nhất
best_model_3 = tuner_3.get_best_models(num_models=1)[0]

# chạy với tập train
best_model_3.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
37/37 [==============================] - 1s 10ms/step - loss: 3560237824.0000 - mean_squared_error: 3560237824.0000 - root_mean_squared_error: 59667.7266 - mean_absolute_error: 43271.2344 - r_squared: 0.2893 - val_loss: 3142924800.0000 - val_mean_squared_error: 3142924800.0000 - val_root_mean_squared_error: 56061.7930 - val_mean_absolute_error: 43411.8945 - val_r_squared: 0.4805
Epoch 2/200
37/37 [==============================] - 0s 4ms/step - loss: 3468951808.0000 - mean_squared_error: 3468951808.0000 - root_mean_squared_error: 58897.8086 - mean_absolute_error: 42510.0859 - r_squared: 0.3067 - val_loss: 3059780352.0000 - val_mean_squared_error: 3059780352.0000 - val_root_mean_squared_error: 55315.2812 - val_mean_absolute_error: 42703.2812 - val_r_squared: 0.4956
Epoch 3/200
37/37 [==============================] - 0s 3ms/step - loss: 3279293184.0000 - mean_squared_error: 3279293184.0000 - root_mean_squared_error: 57265.1133 - mean_absolute_error: 41662.2539 - r_squared: 0

In [26]:
# chạy thử mô hình với tập kiểm tra
best_model_3.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model_3.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model_3.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model_3.metrics[3].result().numpy())
print("R-squared:", best_model_3.metrics[4].result().numpy())

10/10 [==============================] - 0s 3ms/step - loss: 1125877248.0000 - mean_squared_error: 1125877248.0000 - root_mean_squared_error: 33554.0938 - mean_absolute_error: 21036.4102 - r_squared: 0.8577
Mean Squared Error: 1125877200.0
Root Mean Squared Error: 33554.094
Mean Absolute Error: 21036.41
R-squared: 0.8576797


### Kết luận

Sau khi sử dụng Hyperband để cải tiến mô hình:

- Mean Squared Error: 36704793000.0
- Root Mean Squared Error: 191584.95
- Mean Absolute Error: 171676.36
- R-squared: -5.2391396

## 7 -  Chạy thử với tập Test.csv trên model tốt nhất


In [27]:
test_data = pd.read_csv("kaggle/test.csv")
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [28]:
X2=test_data[features]
X2.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,TotRmsAbvGrd,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20,11622,5,6,1961,1961,896,0,0,896,...,5,140,0,0,0,120,0,0,6,2010
1,20,14267,6,6,1958,1958,1329,0,0,1329,...,6,393,36,0,0,0,0,12500,6,2010
2,60,13830,5,5,1997,1998,928,701,0,1629,...,6,212,34,0,0,0,0,0,3,2010
3,60,9978,6,6,1998,1998,926,678,0,1604,...,7,360,36,0,0,0,0,0,6,2010
4,120,5005,8,5,1992,1992,1280,0,0,1280,...,5,0,82,0,0,144,0,0,1,2010


In [29]:
X2 = scaler.transform(X2)
# Dự đoán giá nhà trên tập test.csv với best_model_2 có số R-squared tốt nhất
predictions = best_model_2.predict(X2)

# In ra 10 giá nhà dự đoán đầu tiên
print(predictions[0:10])



46/46 [==============================] - 0s 1ms/step
[[130110.11]
 [160910.64]
 [161390.44]
 [192436.69]
 [208523.11]
 [181726.8 ]
 [182267.52]
 [166771.92]
 [212209.3 ]
 [112506.03]]
